In [ ]:
import numpy as np
import scipy
import scipy.io
import scipy.sparse as sp
import csv

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")

%load_ext autoreload
%autoreload 2
%matplotlib inline

from helpers import calculate_mse

# 0.1 Load and prepare data

### Training and testing data

#### Load the Data
`ratings` is a sparse matrix in the shape of (num_items, num_users)

In [ ]:
from helpers import load_data, preprocess_data

DATA_PATH = '../data/'
PREDICTION_PATH = '../data/predictions/'
ratings = load_data('{dp}data_train.csv'.format(dp=DATA_PATH))
#ratings = load_data('{dp}movielens100k.csv'.format(dp=DATA_PATH))

#### Plot the number of ratings per movie and user

In [ ]:
from plots import plot_raw_data

num_items_per_user, num_users_per_item = plot_raw_data(ratings)

print("min # of items per user = {}, min # of users per item = {}.".format(
        min(num_items_per_user), min(num_users_per_item)))

#### Split the data into a train and test set

In [ ]:
def split_data(ratings, num_items_per_user, num_users_per_item,
               min_num_ratings, p_test=0.1, verbose=False):
    """split the ratings to training data and test data.
    Args:
        min_num_ratings: 
            all users and items we keep must have at least min_num_ratings per user and per item. 
    """
    # set seed
    np.random.seed(988)
    
    # select user and item based on the condition.
    valid_users = np.where(num_items_per_user >= min_num_ratings)[0]
    valid_items = np.where(num_users_per_item >= min_num_ratings)[0]
    valid_ratings = ratings[valid_items, :][:, valid_users]
    
    # LIL is a convenient format for constructing sparse matrices
    train = sp.lil_matrix(valid_ratings.shape)
    test = sp.lil_matrix(valid_ratings.shape)
    
    valid_ratings_i, valid_ratings_u, valid_ratings_v = sp.find(valid_ratings)
    valid_ratings_p_idx = np.random.permutation(range(len(valid_ratings_i)))
    
    n_test = int(p_test*len(valid_ratings_i))
    
    for idx in valid_ratings_p_idx[:n_test]:
        test[valid_ratings_i[idx], valid_ratings_u[idx]] = valid_ratings_v[idx]
        
    for idx in valid_ratings_p_idx[n_test:]:
        train[valid_ratings_i[idx], valid_ratings_u[idx]] = valid_ratings_v[idx]

    if verbose:
        print("Total number of nonzero elements in original data:{v}".format(v=ratings.nnz))
        print("Total number of nonzero elements in train data:{v}".format(v=train.nnz))
        print("Total number of nonzero elements in test data:{v}".format(v=test.nnz))
    
    # convert to CSR for faster operations
    return valid_ratings, train.tocsr(), test.tocsr()

In [ ]:
valid_ratings, train, test = split_data(ratings, num_items_per_user,
    num_users_per_item, min_num_ratings=10, p_test=0.1, verbose=True)

### Read submission creation data

In [ ]:
ratings_csr = ratings.tocsr()
sample_submission = load_data('{dp}sample_submission.csv'.format(dp=DATA_PATH))
sample_submission_csr = sample_submission.tocsr()

# 0.2 Auxiliary functions

In [ ]:
def compute_division(a, b):
    """Computes element by element division.
    If x/0 returns 0.
    """
    # Raises error if vectors have different lengths
    assert(len(a) == len(b))
    
    # Computes division
    res = a.copy()
    for i in range(len(a)):
        if b[i] == 0:
            res[i] = 0
        else:
            res[i] = a[i] / b[i]

    return res

# 1 Baselines

In [ ]:
# Baseline rating
def baseline_rating(data):
    """Implements baseline method for a ratings matrix
    using the global mean.
    """
    # Compute global mean using training data
    r_mean = data.sum() / data.getnnz()
    
    return r_mean


# User or item specific effect
def baseline_user_item_specific(data, mean, set_num=0):
    """Implements baseline method for a ratings matrix
    using either the user or the item mean,
    as indicated in parameter mean.
    """
    if mean=="user":
        flag = 1
        inv_flag = 0
    else:
        flag = 0
        inv_flag = 1

    num = max(set_num, data.shape[flag])
    
    # Obtain r_demeaned (ratings minus global avg)
    global_mean = baseline_rating(data)
    r_demeaned = data.copy()
    r_demeaned.data = (1.0 * r_demeaned.data) - global_mean
    
    # Compute means using training data
    # get rows, columns and values for elements in r_demeaned
    data_rcv = sp.find(r_demeaned)
    # compute means
    counts = np.bincount(data_rcv[flag], minlength=num)
    sums = np.bincount(data_rcv[flag], weights=data_rcv[2], minlength=num)
    means = compute_division(sums, counts)

    return means

In [ ]:
def demean_matrix(data, verbose=False):
    """Removes the global, user and item means from a matrix.
    Returns the matrix and the computed means.
    """
    num_items, num_users = data.shape
    (rows, cols, vals) = sp.find(data)
    
    # Compute global, user and item means    
    global_mean = baseline_rating(data)
    item_means = baseline_user_item_specific(data, 'item')
    user_means = baseline_user_item_specific(data, 'user')
    
    # Substract the baseline of each element in 'data'
    train_vals = vals.copy()
    train_vals = 1.0 * train_vals
    
    baselines = np.array([(global_mean + item_means[i] + user_means[u])
        for (i, u) in zip(rows, cols)])
    train_vals = train_vals - baselines

    # Get matrix
    r_demeaned = sp.csr_matrix((train_vals, (rows, cols)),
        shape=(num_items, num_users))
    
    if verbose:
        print('---------------------------------------------')
        print('          Completed demean_matrix!           ')
        print('---------------------------------------------')
    
    return r_demeaned, global_mean, user_means, item_means

# 2 Matrix Factorization

In [ ]:
import numpy as np

In [ ]:
def init_MF(data, k):
    """Initializes parameters for Matrix Factorization.
    Assumes 'data' matrix is already demeaned.
    """       
    num_items, num_users = train.shape
    u_features = np.zeros(k, num_users)
    i_features = np.zeros(k, num_items)

    return u_features, i_features

Compute the cost by the method of matrix factorization.

In [ ]:
def compute_error(data, u_features, i_features):
    """Compute RMSE for prediction of nonzero elements."""
    (rows, cols, vals) = sp.find(data)
    preds = np.array([(u_features[:,u].dot(i_features[:,i]))
        for (i, u) in zip(rows, cols)])
    mse = calculate_mse(vals, preds)  
    rmse = np.sqrt(mse / len(vals))
    return rmse

In [ ]:
def matrix_factorization_SGD(data, test_data, test_flag, sub_flag=False,
    k=20, gamma=0.01, gamma_dec=1.2, u_lambda=0.5, i_lambda=0.5, epochs=20,
    int_vals=False, sub_filename="new_submission", verbose=False):
    """Matrix factorization by SGD.

    If 'test_flag' is True, then 'data' should be the training dataset and
    'test_data' the test dataset. In this case sub_flag is ignored.
    
    If 'test_flag' is False and 'sub_flag' is True, then 'data' should be
    the entire ratings dataset and 'test_data' should be a sample submission.
    
    Both 'data' and 'test_data' should be csr sparse matrices."""

    assert test_flag or sub_flag, "Specify a task"
    assert k <= min(data.shape), "k must be smaller than the min dimension of 'data'"
    
    # Set seed
    np.random.seed(988)

    # Demean matrix and get feature vectors
    r_demeaned, global_mean, user_means, item_means = demean_matrix(data, verbose=verbose)
    u_features, i_features = init_MF(r_demeaned, k)
    
    # Get non-zero elements
    (rows, cols, vals) = sp.find(data)
    (test_rows, test_cols, test_vals) = sp.find(test_data)
    
    train_samples = zip(rows, cols)

    if verbose:
        print("Starting matrix factorization with SGD...")

    for e in range(epochs):        
# shuffle the training rating indices
np.random.shuffle(train_samples)
        
        # Reduce learning rate
        gamma /= gamma_dec
        
        for i, u in train_samples:
            # update W_d (item_features[:, d]) and Z_n (user_features[:, n])
            item_info = item_features[:, i]
            user_info = user_features[:, u]
            err = data[i, u] - user_info.T.dot(item_info)
    
            # Get gradient and update
            item_features[:, i] += gamma * (err * user_info - i_lambda * item_info)
            user_features[:, u] += gamma * (err * item_info - u_lambda * user_info)

        rmse = compute_error(data, user_features, item_features, nz_train)
        print("{} - train_RMSE: {}.".format(e, rmse))

    # evaluate the test error
rmse = compute_error(test, user_features, item_features, nz_test)
    print("test RMSE: {}.".format(rmse))

In [ ]:
matrix_factorization_SGD(train, test)

TODO: Use functions defined in ex10 to define the following model

In [ ]:
def model_matrix_factorization(data, test_data, test_flag, sub_flag=False,
    k=20, int_vals=False, sub_filename="new_submission", verbose=False):
    """If 'test_flag' is True, then 'data' should be the training dataset and
    'test_data' the test dataset. In this case sub_flag is ignored.
    
    If 'test_flag' is False and 'sub_flag' is True, then 'data' should be
    the entire ratings dataset and 'test_data' should be a sample submission.
    
    Both 'data' and 'test_data' should be csr sparse matrices.
    """

    assert test_flag or sub_flag, "Specify a task"
    assert k <= min(data.shape), "k must be smaller than the min dimension of 'data'"
    
    # Substract baseline from data
    r_demeaned, global_mean, user_means, item_means = demean_matrix(data, verbose=verbose)
    # Use scipy's svds
    U, sigma, Vt = svds(r_demeaned, k)
    sigma = np.diag(sigma)
    U_sigma = np.dot(U, sigma)
    
    if verbose:
        print('Finished fitting model')

    (test_rows, test_cols, test_vals) = sp.find(test_data)
    (rows, cols, vals) = sp.find(data)

    if test_flag:        
        # Do predictions        
        baselines = np.array([(global_mean + item_means[i] + user_means[u])
            for (i, u) in zip(test_rows, test_cols)])
        interactions = np.array([(U_sigma[i,:].dot(Vt[:,u]))
            for (i, u) in zip(test_rows, test_cols)])
        pred_test = baselines + interactions
        if int_vals:
            pred_test = np.rint(pred_test)
      
        if verbose:
            print('Finished predicting')

        # Compute and print test error
        test_mse = calculate_mse(test_vals, pred_test)
        test_rmse = np.sqrt(test_mse / len(test_vals))
        
        if verbose:
            print("Test RMSE using baseline and matrix factorization: {e}".format(e=test_rmse)) 
            print()
            print('-----------------------------------------------')
            print(' Completed test in model_matrix_factorization! ')    
            print('-----------------------------------------------')
            
        return test_rmse, pred_test

    elif sub_flag:
        # Directly write predictions to submission file
        with open('{dp}{fn}.csv'.format(dp=PREDICTION_PATH, fn=sub_filename), 'w') as csvfile:
            fieldnames = ['Id', 'Prediction']
            writer = csv.DictWriter(csvfile, delimiter=",", fieldnames=fieldnames)
            writer.writeheader()
            for (i, u) in zip(test_rows, test_cols):
                interaction = U_sigma[i,:].dot(Vt[:,u])
                baseline = global_mean + user_means[u] + item_means[i]
                pred_i_u = interaction + baseline
                writer.writerow({'Id':'r{r}_c{c}'.format(r=i+1,c=u+1),'Prediction':pred_i_u})

        if verbose:
            print('-----------------------------------------------------')
            print(' Completed submission in model_matrix_factorization! ')    
            print('-----------------------------------------------------')

Test:

Now we prepare the submission file for Kaggle:

## E.2 Matrix Factorization for Ex10

### Matrix Factorization using SGD

Some guy wanted to optimize running time, this is his code:

https://codereview.stackexchange.com/questions/35727/optimize-scipy-sparse-matrix-factorization-code-for-sgd

In [ ]:
N = self.model.shape[0] #no of users
M = self.model.shape[1] #no of items
self.p = np.random.rand(N, K)
self.q = np.random.rand(M, K)
rows,cols = self.model.nonzero()        
for step in xrange(steps):
    for u, i in zip(rows,cols):
        e = self.model[u, i] - np.dot(self.p[u, :], self.q[i, :]) #calculate error for gradient
        p_temp = learning_rate * ( e * self.q[i,:] - regularization * self.p[u,:])
        self.q[i,:]+= learning_rate * ( e * self.p[u,:] - regularization * self.q[i,:])
        self.p[u,:] += p_temp